- AI 에이전트
  + 사용자의 목표를 달성하기 위해 자율적으로 환경을 인지하고, 데이터를 처리하며, 의사결정을 내려 실제 행동을 수행하는 인공지능 시스템
  + 특징 : 자율성, 인지, 추론,계획, 행동, 협력
    + 챗봇 : 주어진 질문에 답변하는 역할
    + AI 에이전트 : 업무 실행, 프로세스 자동화, 의사결정 까지 수행
  + 예시 : 여행 계획(Trip Planner)
    + 사용자 입력: 출발지, 목적지, 여행 시기, 선호 활동, 예산
    + 에이전트 A: 항공편/교통편 조사
    + 에이전트 B: 숙박 및 일정 추천
    + 에이전트 C: 활동·관광지 일정 구성 → 맞춤형 여행 일정표 자동 생성  
  + 프레임웍 : LangGraph, CrewAI, Open AI Agents, ....  

- GPT tools 
  - 여러 tool 을 정의하면 GPT 는 상황에 맞게 선택하거나 동시에 호출해서 문제를 해결합니다. AI 에이전트 구현에 사용.
  - tools : (function_call은  단일 함수만 호출 합니다.)
    + gpt-4 모델 부터 도입.
    + 여러 함수 또는 API 등등을 동시에 호출.
    + 리턴값 중 tool_calls 항목은 배열(리스트)

In [1]:
!python ai_responseV3.py

In [9]:
from openai import OpenAI
from dotenv import load_dotenv
from ai_responseV2 import get_current_date_tz,get_current_time_tz
from ai_responseV3 import get_ai_response_tools,get_first_response_tools,get_followup_response_tools
load_dotenv()

client = OpenAI()

#### 1. get_first_response_tools
- get_first_response_tools 에서 사용할 tool_calls 확인

In [3]:
question = '지금 뉴욕의 날짜와 시간을 알려줘'
response = get_first_response_tools(question)
print(type(response))
tool_calls = response.choices[0].message.model_dump_json(indent=2)
print(tool_calls)

<class 'openai.types.chat.chat_completion.ChatCompletion'>
{
  "content": null,
  "refusal": null,
  "role": "assistant",
  "annotations": [],
  "audio": null,
  "function_call": null,
  "tool_calls": [
    {
      "id": "call_hXCfZBRG053sLwur6z8vs5fk",
      "function": {
        "arguments": "{\"timezone\": \"America/New_York\"}",
        "name": "get_current_date_tz"
      },
      "type": "function"
    },
    {
      "id": "call_bEUAMcRp7Bqhs50TqQclvSEd",
      "function": {
        "arguments": "{\"timezone\": \"America/New_York\"}",
        "name": "get_current_time_tz"
      },
      "type": "function"
    }
  ]
}


#### 2. tool_calls 의 함수 이름 문자열로 가져오기
- tool_results 에 tools_calls 에 있는 함수 문자열로 가져와서 저장

In [10]:
import json
tool_results = []
for tool in response.choices[0].message.tool_calls:
    fn_name = tool.function.name
    args = json.loads(tool.function.arguments)
    
    # 함수 이름을 문자열로 가져와서 globals() 이용하여 실행하기
    if fn_name:
        result = globals()[fn_name](**args)
        tool_results.append({"name": fn_name, "result": result})
    else:
        tool_results.append({"name": fn_name, "result": f"Unknown function: {fn_name}"})

현재 날짜(tz) : 2025 년 11 월 06 일 America/New_York
현재 시간(tz) : 02:40:03 America/New_York


##### globals()\[fn_name](**args) 가 함수로 실행되는 이유
- globals() 함수 : 전역 변수들의 변수 이름 및 현재 값을 보여주는 함수
- 1️⃣ fn_name 는 함수 이름을 저장하고 있음
- 2️⃣ globals()[fn_name] 로 전역 변수 중 키값(fn_name)으로 value 를 가져오면 함수 객체 자체를 반환
    - globals()\[fn_name]() 하면 함수가 실행됨
- 3️⃣ (**args) 는 딕셔너리를 풀어서 함수 인자로 넘기는 역할
    - "{\"timezone\": \"America/New_York\"}" -> timezone = America/New York
- 4️⃣ 결과적으로 함수를 실행시는 형태가 됨
    - ai_responseV2.get_current_time_tz(timezone='America/New York')

In [18]:
# 1️⃣
fn_name     # "name": "get_current_date_tz"

'get_current_time_tz'

In [16]:
# 2️⃣
globals()[fn_name]

<function ai_responseV2.get_current_time_tz(timezone='Asia/Seoul')>

In [20]:
# 3️⃣
args    # "arguments": "{\"timezone\": \"America/New_York\"}"

{'timezone': 'America/New_York'}

In [12]:
# 4️⃣
tool_results

[{'name': 'get_current_date_tz', 'result': '2025 년 11 월 06 일'},
 {'name': 'get_current_time_tz', 'result': '02:40:03'}]

##### - 참고
- globals()[func] 을 사용할 때 함수이름을 문자열로 저장해야하는 이유
```
globals() == {
    'func': <class 'int'>,
    'str': <class 'str'>,
    '__name__': '__main__',
}
```
- -> globals() 의 키는 문자열로 저장되기 때문에
- ex) func = int
    - 여기서 func 은 문자열이 아니라 <class 'int'> 라는 객체로 저장됨
    - globals()[func] 는 globals()[<class 'int'>] 로 키에 접근 하는 형태
    - 해당 키를 찾을 수 없어 `KeyError` 발생

#### 3. get_followup_response_tools 함수 실행해서 확인

In [13]:
get_followup_response_tools(question,tool_results)

'현재 뉴욕의 날짜는 2025년 11월 6일이며, 시간은 02시 40분 3초입니다.'

#### 4. 1 ~ 3 번을 한번에 실행할 수 있는 get_ai_response_tools 실행

In [4]:
print(get_ai_response_tools(question))

현재 날짜(tz) : 2025 년 11 월 06 일 America/New_York
현재 시간(tz) : 02:35:51 America/New_York
지금 뉴욕의 날짜는 2025년 11월 6일이며, 시간은 02시 35분 51초입니다.
